In [1]:
from typing import List, Any, Sequence
from numpy import arange
from sinergym.utils.common import get_season_comfort_range
import gym
import datetime
import numpy as np
import sinergym

In [2]:
from sinergym.utils.controllers import RuleBasedController

class MyRuleBasedController(RuleBasedController):

    def act(self, observation: List[Any]) -> Sequence[Any]:
        """Select action based on outdoor air drybulb temperature and daytime.

        Args:
            observation (List[Any]): Perceived observation.

        Returns:
            Sequence[Any]: Action chosen.
        """
        obs_dict = dict(zip(self.variables['observation'], observation))

        out_temp = obs_dict['Site Outdoor Air Drybulb Temperature (Environment)']

        day = int(obs_dict['day'])
        month = int(obs_dict['month'])
        hour = int(obs_dict['hour'])

        season_comfort_range = get_season_comfort_range(month, day)

        if out_temp in arange(
                season_comfort_range[0], season_comfort_range[1], .1):
            action = (obs_dict[
                'Zone Thermostat Heating Setpoint Temperature (SPACE1-1)'],
                    obs_dict[
                'Zone Thermostat Cooling Setpoint Temperature (SPACE1-1)'])
        elif datetime.date(day=day, month=month).weekday() > 5:
            action = (10.0, 30.0)
        elif hour in range(6, 18):  # day
            action = (19.44, 25.0)
        elif hour in range(18, 22):  # evening
            action = (20.0, 24.44)
        else:  # night
            action = (18.33, 23.33)

        return action

In [3]:
env = gym.make('Eplus-demo-v1')

# create rule-controlled agent
agent = MyRuleBasedController(env)

for i in range(1):
    obs = env.reset()
    rewards = []
    done = False
    current_month = 0
while not done:
    action = agent.act(obs)
    obs, reward, done, info = env.step(action)
    rewards.append(reward)
    if info['month'] != current_month:  # display results every month
        current_month = info['month']
        print('Reward: ', sum(rewards), info)

[2022-04-11 14:17:48,969] EPLUS_ENV_5Zone-mixed-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-04-11 14:17:48,971] EPLUS_ENV_5Zone-mixed-continuous-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
C:\Users\Melon\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
[2022-04-11 14:17:48,975] EPLUS_ENV_5Zone-mixed-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...


UnicodeEncodeError: 'charmap' codec can't encode character '\ufffd' in position 1063: character maps to <undefined>